In [1]:
# 1D chain of spinless atoms with infinite Coulomb repulsion

In [2]:
import numpy as np
import numpy.linalg as LA
from tqdm import tqdm

import time
import matplotlib.pyplot as plt

In [3]:
# parameters

Ns = 20   # number of sites
Np = 7   # number of particles, filling = Np/Ns

MCsteps = int(1.e5)

burn = 1.e4     # how many steps to burn
sweep = 1.e3    # sample configs after what steps

In [4]:
def pbc(n):
    if n<0:
        return n+Ns
    elif n>=Ns:
        return n-Ns
    else:
        return n

In [5]:
# single particle states

all_sp_states = np.arange(-np.pi, np.pi, 2*np.pi/Ns)
energy = all_sp_states*all_sp_states
idx = energy.argsort()[:]
all_sp_states_energy_sorted = all_sp_states[idx]

In [6]:
# for our problem, we only need Np of the lowest states
sp_states = all_sp_states_energy_sorted[0:Np]

In [7]:
# particle positions

def slaterPsi(particle_positions):

    # slater determinant
    slater = np.zeros([Np,Np]).astype(complex)

    for i in range(Np):
        for j in range(Np):
            ki = sp_states[i]
            rj = particle_positions[j]
            slater[i,j] = np.exp(1j*ki*rj)
    
    return np.asmatrix(slater)

def jasPsi(particle_positions,g):
    # jastrow factor
    jastrow = 1.0
    for p1 in particle_positions:
        ngbr = [pbc(p1-1),pbc(p1+1)]
        if ngbr in particle_positions:  # neareast neighbors
            jastrow = jastrow*g

    return jastrow


In [8]:
def Dprime(D0, detD0, D0bar, new_config, part_index):
    # gives detD1, D1bar as output -- *without* calculating det(D1) and inv(D1).T

    j0 = part_index     # particle that has moved

    D1 = slaterPsi(new_config)

    qRatio = np.dot( D1[:,j0].T,D0bar[:,j0] )[0,0]
    detD1 = qRatio*detD0

    DbarAr = D0bar[:,j0]
    M = (D0bar.T)*(D1-D0)

    D1bar = D0bar - np.outer(D0bar[:,j0],M[:,j0])/qRatio

    return detD1, D1bar

In [9]:
# now let's do a MC step

# start with random position

def GiveList(g):

    rij_list = []
    d0_list = []
    detD0_list = []
    d0bar_list = []

    pos = np.random.choice(Ns, Np, replace=False)   # starting position

    jas0 = jasPsi(pos,g)
    d0 = slaterPsi(pos)
    d0bar = LA.inv(d0).T
    detD0 = LA.det(slaterPsi(pos))

    for step in range(MCsteps):
        
        prob = (jas0*np.abs(detD0))**2
    
        # select a particle at random
        pp_rand = np.random.randint(Np)
        # change its position
        new_pos = np.copy(pos)
        # give it a kick
        kick = np.random.choice(np.setdiff1d(np.arange(Ns),pos))
        # update the position
        new_pos[pp_rand] = kick

        detD1, d1bar = Dprime(d0, detD0, d0bar, new_pos, pp_rand)
        jas1 = jasPsi(new_pos,g)

        prob_new = (jas1*np.abs(detD1))**2

        # is it any better?
        ratio = prob_new/prob

        # make the move
        if (ratio > 1):
            pos = new_pos
            prob = prob_new
            detD0 = detD1
            d0bar = d1bar
            d0 = slaterPsi(new_pos)
#            acpr += 1.0/MCsteps
        else:
            r = np.random.random()  # metropolis step
            if (r < ratio ):
                pos = new_pos
                prob = prob_new
                detD0 = detD1
                d0bar = d1bar
                d0 = slaterPsi(new_pos)     # acpr += 1.0/MCsteps     # acceptance ratio

        if (step > burn and step%sweep ==0):
            rij_list.append(pos)
            detD0_list.append(detD0)
            d0_list.append(d0)
            d0bar_list.append(d0bar)

    return rij_list, detD0_list, d0_list, d0bar_list
#print('acceptance ratio', acpr)
    

In [10]:
rij, detlist, d0list, d0barlist = GiveList(0.0);

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:21: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.


In [11]:
for mc in rij:
    print(mc)

[ 3 18 13 10  8  6 19]
[ 3  4 13 11  8  6 19]
[ 3 15 13 11  8  2 19]
[ 3 17 13  5  8 10 19]
[ 3  0 13  5  8 10 19]
[ 3 14 13  5  8 10 19]
[ 3  4 13  5  8 10 19]
[ 3 14 13  6  8 10 19]
[ 3  7 18  4  8 10 19]
[ 3  6 18  4  8 10 19]
[ 3 12 18 19  8 10  6]
[ 3 14 18 19  8  2  6]
[ 3 10 18 19  8  2 15]
[ 3  9 18 19  8  2 15]
[ 3  5 18 19  8  2 15]
[ 3 14 18 19  8  2 15]
[ 3 12 18 19  8  2 15]
[ 3  0 18 19  8  2 13]
[ 3  6 18 19  8  2 13]
[ 3 10 18 19  8  2 13]
[ 3 16 18 19  8  2 13]
[ 3  6 18 19  8  2 13]
[ 3  7 18 19  8  2 13]
[ 3  7 18 19  8  2 13]
[ 3 16 18 19  8  2 13]
[ 3 12 18 19  8  2 13]
[ 3 11 18 19  8  2 13]
[ 3 17 18 19  8  2 13]
[ 3  0 18 19  8  2 13]
[ 3  7 18 19  8  2 13]
[ 3  5 18 19  8  2 13]
[ 3 14 18 19  8  2 13]
[ 3 12 18 19  8  2 13]
[ 3 14 18 19  8  2 13]
[ 3 15 18 19  8  2 13]
[ 3  1 18 19  8  2 13]
[ 6 17 18 19  8  2 13]
[ 6 12 18 19  8  2 13]
[ 6  7 18 19  8  2 13]
[ 6 10 18 19  8  2 13]
[ 6 15 18 19  8  2 13]
[ 6 10 18 19  8  2 13]
[ 6 10 18 19  8  2 13]
[ 6 17 18 1

In [12]:
# observables
# given a configuration, how do you calculate total Energy?

def PEconfig(config):
    pec = 0.0
    for pos in config:
        if np.isin(pbc(pos-1),config):
            pec += 1.0
        if np.isin(pbc(pos+1),config):
            pec += 1.0
    
    return pec

def KEconfig(config, detD0, d0, d0bar,g):
    kec = 0.0
    jas0 = jasPsi(config,g)
    for particle in range(Np):
        new_config = np.copy(config)
        new_config[particle] = pbc(config[particle]+1)
        #print(config, new_config, particle,kec,'chk\n')
        detD1, d1bar = Dprime(d0, detD0, d0bar, new_config, particle)
        jas1 = jasPsi(new_config,g)
        kec += (detD1/detD0)*(jas1/jas0)

        new_config = np.copy(config)
        new_config[particle] = pbc(config[particle]-1)
        #print(config, new_config, particle,kec,'chk\n')
        detD1, d1bar = Dprime(d0, detD0, d0bar, new_config, particle)
        jas1 = jasPsi(new_config,g)
        kec += (detD1/detD0)*(jas1/jas0)

    return kec

In [13]:
#rij, detlist, d0list, d0barlist = GiveList(0.0)

#KEconfig(rij[0], detlist[0], d0list[0], d0barlist[0], 0.0)

In [14]:
# let us calculate energy for g=1, U=0 (no PE)

U = 0.0
g = 0.0

# get MC configs
#rijlist, detlist, d0list, d0barlist = GiveList(g)

KE = []
rijlist=rij
for i in tqdm(range(len(rijlist))):
    config = rijlist[i]
    detD0 = detlist[i]
    d0 = d0list[i]
    d0bar = d0barlist[i]
    KE.append( KEconfig(config, detD0, d0, d0bar,g) )

100%|██████████| 89/89 [00:00<00:00, 129.37it/s]


In [15]:
KE

[(2.093619438396632-0.09815985894407867j),
 (2.108248801747545+0.018257017989075877j),
 (2.1374630460155752-0.019892100608771633j),
 (2.119756988193334+0.055926845075863554j),
 (2.0274011280097457+0.03588483331254521j),
 (2.020062494799586+0.021995021034006905j),
 (2.0632853737195456-0.21236988750157723j),
 (2.008012836890296-0.0008944401953001821j),
 (1.9876320085453965-0.22065913733395004j),
 (1.9852884313182553-0.2111150791979912j),
 (1.9573867026172034-0.09023461204127617j),
 (1.911818709606403-0.22742789831764149j),
 (2.0457014884558173-0.0561196372603594j),
 (2.0215114714297844-0.06766100948002104j),
 (1.9965795190884348-0.08182480953821046j),
 (2.0270367409100936-0.02146954123132923j),
 (2.055143943343454-0.04518625370022335j),
 (2.0115771708527674+0.00942765581640812j),
 (1.97579080296093-0.046454397217841226j),
 (2.0298032389090905+0.0016384372312255814j),
 (2.039391625331007-0.007711855387104286j),
 (1.975790802960934-0.04645439721784129j),
 (1.9741566279564637-0.046915321474

In [16]:
# let us calculate energy for g=1, U=0 (no PE)

dg = 0.01
gar = np.arange(0,1+dg,dg)

KE_g = []
PE_g = []

for g in tqdm(gar):
    rijlist, detlist, d0list, d0barlist = GiveList(g)
    KE = []
    PE = []
    for i in range(len(rijlist)):
        config = rijlist[i]
        detD0 = detlist[i]
        d0 = d0list[i]
        d0bar = d0barlist[i]
        KE.append( KEconfig(config, detD0, d0, d0bar,g) )
        PE.append( PEconfig(config) )


    KE_g.append(np.mean(KE))
    PE_g.append(np.mean(PE))

 62%|██████▏   | 63/101 [1:15:46<45:42, 72.17s/it]


KeyboardInterrupt: 

In [17]:
PE_g = np.asarray(PE_g)
KE_g = np.asarray(KE_g)

np.save('data/pe_g',PE_g)
np.save('data/ke_g',KE_g)